In [57]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings("ignore")

In [58]:
df1=pd.read_csv("E:/EXCELR/01_PROJECTS/P186/Model_Building/New_Trail/Cleaned_Data_Actual.csv",
               dtype={"User-ID": int, "Book-Title": "string",'Book-Rating':int,'Age':int,
                     'Country':'string','Num of ratings':int,'Avg_Rating':float,'Books_Read':int})
dfu=pd.read_csv("E:/EXCELR/01_PROJECTS/P186/Model_Building/New_Trail/User_USA.csv",
               dtype={"User-ID": int, "Book-Title": "string",'Book-Rating':int,'Age':int,
                     'Country':'string','Num of ratings':int,'Avg_Rating':float,'Books_Read':int})
dfo=pd.read_csv("E:/EXCELR/01_PROJECTS/P186/Model_Building/New_Trail/User_Other.csv",
               dtype={"User-ID": int, "Book-Title": "string",'Book-Rating':int,'Age':int,
                     'Country':'string','Num of ratings':int,'Avg_Rating':float,'Books_Read':int})

In [80]:
df2=df1[df1['Book-Rating']!=0]
dfo=df2[df2['Country']!='usa']

In [81]:
dfo.head(2)

,Book-Title,User-ID,Book-Rating,Age,Country,Avg_Rating,Num of ratings,Books_Read
1,Clara Callan,8,5,35,canada,4.928571,14,18
9,Where You'll Find Me: And Other Stories,8,5,35,canada,2.500000,2,18


In [82]:
df_r=pd.DataFrame(dfo.groupby('Book-Title')['Book-Rating'].mean())
df_r=df_r.rename(columns={'Book-Rating':'Avg_Rating'})
df_r['Num of ratings']=pd.DataFrame(dfo.groupby('Book-Title')['Book-Rating'].count())
df_r.shape

(64533, 2)

In [83]:
dfo=dfo.drop(columns={'Avg_Rating','Num of ratings'})
dfo.head(2)

,Book-Title,User-ID,Book-Rating,Age,Country,Books_Read
1,Clara Callan,8,5,35,canada,18
9,Where You'll Find Me: And Other Stories,8,5,35,canada,18


In [84]:
dfo=pd.merge(dfo,df_r,on='Book-Title',how='left')

In [85]:
dfo.dropna(inplace=True)

In [86]:
dfo.to_csv('E:/EXCELR/01_PROJECTS/P186/Model_Building/New_Trail/User_Other.csv', sep=',', header=True, index=False, mode='w')

In [3]:
#To find row of given User-ID
#df1.loc[df1['User-ID'] == 257187]

In [4]:
#dfo.to_csv('E:/EXCELR/01_PROJECTS/P186/Model_Building/New_Trail/User_Other.csv', sep=',', header=True, index=False, mode='w')

In [5]:
df2=df1
df2=df2.drop_duplicates(keep='first')
dfu=dfu.drop_duplicates(keep='first')
dfo=dfo.drop_duplicates(keep='first')

In [6]:
df2 = df2[df2['Book-Rating'] != 0]

#  Books received atleast N feedbacks/ratings

In [7]:
min_rat1 = 50
min_rat2 = 10

# Get a count of the number of ratings per user
book_ratings1 = dfu.groupby("Book-Title")["Book-Rating"].count()

# Keep only users who have rated at least min_ratings books
df3 = dfu[dfu["Book-Title"].isin(book_ratings1[book_ratings1 >= min_rat1].index)]
print("Number of Unique users:",df3['User-ID'].nunique())
print("Number of unique Book-Titles",df3['Book-Title'].nunique())

# Get a count of the number of ratings per user
book_ratings2 = dfo.groupby("Book-Title")["Book-Rating"].count()

# Keep only users who have rated at least min_ratings books
df4 = dfo[dfo["Book-Title"].isin(book_ratings2[book_ratings2 >= min_rat2].index)]
print("Number of Unique users:",df4['User-ID'].nunique())
print("Number of unique Book-Titles",df4['Book-Title'].nunique())

Number of Unique users: 16886
Number of unique Book-Titles 445
Number of Unique users: 9004
Number of unique Book-Titles 1197


# Create DataFrame "df_user" that has Books read by particular User-ID

In [8]:
#Choose UID 7346,16795,...... from above table
#They rated more number of books, so suggesting book would be feasible
UID=int(input('Enter User-ID:'))

Enter User-ID:35704


In [9]:
if UID in df3['User-ID'].values:
    df_user=df3[df3['User-ID']==UID]
elif UID in df4['User-ID'].values:
    df_user=df4[df4['User-ID']==UID]
elif UID in df2['User-ID'].values:
    df_user=df2[df2['User-ID']==UID]
    country=set(df_user['Country'])
    country_df = df2[df2['Country'] == country]
    country_df = country_df.sort_values(by=['Book-Rating','Num of ratings','Avg_Rating'],ascending=[False,False,False])
    list2=country_df['Book-Title'][:300]
    df_user=df2[df2['User-ID']==UID]
    list1=list(df_user['Book-Title'])
    reco_a=[x for x in list2 if x not in list1]
    reco=reco_a[:10]
else:
    df_temp=df1.sort_values(by=['Book-Rating','Num of ratings','Avg_Rating'],ascending=[False,False,False])
    reco=df_temp['Book-Title'][:10]

In [10]:
#Sort Books by ratings
df_user.sort_values('Book-Rating',ascending=False,inplace=True)
df_user.head()

,Book-Title,User-ID,Book-Rating,Age,Country,Books_Read,Avg_Rating,Num of ratings
45,Nickel and Dimed: On (Not) Getting By in America,35704,10,53,usa,88,8.231788,151
762,Stupid White Men ...and Other Sorry Excuses fo...,35704,10,53,usa,88,7.834862,109
196,The Da Vinci Code,35704,9,53,usa,88,8.471033,397
683,Naked,35704,8,53,usa,88,8.050633,79
891,Jurassic Park,35704,7,53,usa,88,7.950000,160


In [11]:
#Get "Book-Title" which is  MOST LIKED by given User-ID
k=df_user['Book-Title'].iloc[0]
print(k)

Nickel and Dimed: On (Not) Getting By in America


# Build Pivot Table to determine Item Similarity

In [12]:
df3=df3.sort_values('User-ID')

In [13]:
df6=df3.pivot_table(index="User-ID",columns='Book-Title',values='Book-Rating')
df6.head()

Book-Title,1984,1st to Die: A Novel,2nd Chance,A Bend in the Road,A Case of Need,"A Child Called \It\"": One Child's Courage to S...",A Confederacy of Dunces (Evergreen Book),A Heartbreaking Work of Staggering Genius,A Is for Alibi (Kinsey Millhone Mysteries (Pap...,A Lesson Before Dying (Vintage Contemporaries ...,...,Wicked: The Life and Times of the Wicked Witch...,Wild Animus,Winter Solstice,Wish You Well,Without Remorse,"Wizard and Glass (The Dark Tower, Book 4)",Wuthering Heights,Year of Wonders,Zen and the Art of Motorcycle Maintenance: An ...,"\O\"" Is for Outlaw"""
User-ID,,,,,,,,,,,,,,,,,,,,,
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#Impute NaN values with Zero as it won't affect while computing Cosine Similarity or Euclidean Distance
#This is verified at end during Model Evaluation
df6.fillna(0,inplace=True)

In [15]:
df6.head()

Book-Title,1984,1st to Die: A Novel,2nd Chance,A Bend in the Road,A Case of Need,"A Child Called \It\"": One Child's Courage to S...",A Confederacy of Dunces (Evergreen Book),A Heartbreaking Work of Staggering Genius,A Is for Alibi (Kinsey Millhone Mysteries (Pap...,A Lesson Before Dying (Vintage Contemporaries ...,...,Wicked: The Life and Times of the Wicked Witch...,Wild Animus,Winter Solstice,Wish You Well,Without Remorse,"Wizard and Glass (The Dark Tower, Book 4)",Wuthering Heights,Year of Wonders,Zen and the Art of Motorcycle Maintenance: An ...,"\O\"" Is for Outlaw"""
User-ID,,,,,,,,,,,,,,,,,,,,,
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 1. Cosine Similarity

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
cosine_sim1 = cosine_similarity(df6)

In [46]:
cosine_sim1

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [47]:
#Converting array in DataFrame
user_sim_df=pd.DataFrame(cosine_sim1)
user_sim_df.index = df3['User-ID'].unique()
user_sim_df.columns = df3['User-ID'].unique()
user_sim_df.head()


,9,16,26,32,42,51,97,99,114,125,...,278773,278798,278800,278824,278828,278832,278836,278843,278844,278854
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
user_sim_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16886 entries, 9 to 278854
Columns: 16886 entries, 9 to 278854
dtypes: float64(16886)
memory usage: 2.1 GB


In [54]:
user_sim_df.columns = [str(col) for col in user_sim_df.columns]
user_sim_df.to_parquet('USA_sim1.parquet.gzip', compression='gzip')

In [55]:

# Import from Parquet
df_utemp = pd.read_parquet('USA_sim1.parquet.gzip')
df_utemp.columns=[int(col) for col in df_utemp.columns]
df_utemp.head()

,9,16,26,32,42,51,97,99,114,125,...,278773,278798,278800,278824,278828,278832,278836,278843,278844,278854
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
df_utemp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16886 entries, 9 to 278854
Columns: 16886 entries, 9 to 278854
dtypes: float64(16886)
memory usage: 2.1 GB


In [44]:
user_sim_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16886 entries, 9 to 278854
Columns: 16886 entries, 9 to 278854
dtypes: float64(16886)
memory usage: 2.1 GB


In [42]:
#user_sim_df.to_csv('E:/EXCELR/01_PROJECTS/P186/Deployment/USApivot50.csv', sep=',', header=True, index=False, mode='w')

In [23]:
#Select the row from similarity matrix as per the given User-ID
user_simi = user_sim_df.loc[UID, :]
#Sort the above variable and get top-5 users similar to our defined User
top_5 = user_simi.sort_values(ascending=False)[1:6]
simi_users = top_5.index.tolist()
simi_users

[156816, 232777, 48430, 240370, 234623]

In [24]:
#Create empty dataframe
df_reco=pd.DataFrame(columns=df3.columns)
df_reco.head(2)

,Book-Title,User-ID,Book-Rating,Age,Country,Books_Read,Avg_Rating,Num of ratings


In [25]:
#Create Dataframe with User-ID similar to given User-ID
for i in simi_users:
    entry = df3.loc[df3['User-ID'] == i]
    df_reco=pd.concat([df_reco,entry])

df_reco.drop_duplicates(keep='first',inplace=True)

In [26]:
df_reco1=df_reco[(df_reco['Book-Rating']>=5) ].sort_values(by='Book-Rating',ascending=False)
df_reco1.dropna(inplace=True)
print(df_reco1.shape)

(15, 8)


# Model Evaluation

In [27]:
list1=list(df_user['Book-Title'])
print("Number of books read by User-",UID,":",df_user['Book-Title'].nunique())

Number of books read by User- 35704 : 5


In [28]:
list2=list(df_reco1['Book-Title'])
print("Number of books recommended based on Top-5 similar users:",df_reco1['Book-Title'].nunique())

Number of books recommended based on Top-5 similar users: 8


In [29]:
list3=list(df_user['Book-Title'][:10])

In [30]:
list4=list(df_reco1['Book-Title'][:10])

In [31]:
reco=[x for x in list2 if x not in list1]

In [32]:
comp1=[x for x in list4 if x in list3]
print("Books read by user which are in recommended list",len(comp1))

Books read by user which are in recommended list 9


In [33]:
comp=[x for x in list2 if x in list1]
print("Books read by user which are in recommended list",len(comp))

Books read by user which are in recommended list 11


In [34]:
#Creating dataframe which can used in deployment
reco1=pd.DataFrame({'Recommendation Books': reco})
reco1.head()

,Recommendation Books
0,Fast Food Nation: The Dark Side of the All-Ame...
1,Zen and the Art of Motorcycle Maintenance: An ...
2,Fahrenheit 451
3,A Confederacy of Dunces (Evergreen Book)


# Book Recommendation for given User-ID

In [35]:
print("The Top-10 recommended books are :",'\n')
for i in reco[:10]:
    print(i)

The Top-10 recommended books are : 

Fast Food Nation: The Dark Side of the All-American Meal
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values
Fahrenheit 451
A Confederacy of Dunces (Evergreen Book)


In [36]:
acc=round(len(comp1)*100/10,2)
print('The Accuracy of Model for User-ID:',UID,' is :',acc)

The Accuracy of Model for User-ID: 35704  is : 90.0
